In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import time
import pickle
import collections
import string
import warnings
warnings.filterwarnings('ignore')

In [2]:
from random import seed
RANDOM = 42
seed(RANDOM)
np.random.seed(RANDOM)

In [3]:
videos_df = pd.read_csv("videos.csv")
comments_df = pd.read_csv("comments.csv")

In [4]:
video_id_to_category = {}
for _, row in videos_df.iterrows():
    if row['video_id'] not in video_id_to_category:
        video_id_to_category[row['video_id']] = row['category_id']

In [5]:
video_id_to_category

{'XpVt6Z1Gjjo': 24,
 'K4wEI5zhHB0': 28,
 'cLdxuaxaQwc': 22,
 'WYYvHb03Eog': 28,
 'sjlHnJvXdQs': 23,
 'cMKX2tE5Luk': 1,
 '8wNr-NQImFg': 23,
 '_HTXMhKWqnA': 28,
 '_ANP3HR1jsM': 22,
 'zgLtEob6X-Q': 1,
 'Ayb_2qbZHm4': 23,
 'CsdzflTXBVQ': 23,
 'l864IBj7cgw': 23,
 '4MkC65emkG4': 24,
 'vu_9muoxT50': 24,
 '1L7JFN7tQLs': 28,
 'ZQK1F0wz6z4': 24,
 'T_PuZBdT2iM': 1,
 'w8fAellnPns': 26,
 'UCrBICYM0yM': 1,
 '-Ifnaxi2LQg': 10,
 'B7YaMkCl3XA': 25,
 '5ywKal6-anc': 24,
 '4Yue-q9Jdbk': 24,
 'JhA1Wi9mrns': 10,
 'EVp4-qjWVJE': 17,
 'LcZ2AuvxXNA': 17,
 'MdzGZv3zQ-U': 26,
 '1ZNZY-gd3K0': 24,
 '2wxyDrfwlXQ': 26,
 'QBGaO89cBMI': 10,
 'ql0Op1VcELw': 27,
 'JO7X9ZPoAp8': 24,
 'GGm0FQ6i74U': 25,
 'a7Sf_H2cFdM': 1,
 'oDIDZ9EmQfA': 24,
 'zAtHxJvSczA': 24,
 'l_uNfGY3v8E': 25,
 'Rew50oYjRqA': 26,
 'WWexI9YiLSc': 1,
 'LDcm6twPEJA': 27,
 'O78Lpo4ctSE': 10,
 'qJJHhVf3_ZM': 24,
 '3yoGE3v4A8w': 26,
 'zTjcPeb2Gwg': 22,
 'hQbeB0YXLhs': 26,
 'L3f7_y9UPh4': 23,
 'HwYK5Ay1YNQ': 25,
 '6vGg-jJl30A': 17,
 '9CILSvf1snQ': 26,
 'WoPt

In [6]:
comments_df['category_id'] = comments_df.apply(lambda row: video_id_to_category[row['video_id']], axis=1)
le = preprocessing.LabelEncoder()
le.fit(comments_df['category_id'])
comments_df['category_id'] = le.transform(comments_df['category_id']) 

In [7]:
comments_df.groupby(['category_id']).agg(['count'])

,video_id,comment_text,likes,replies
,count,count,count,count
category_id,,,,
0,65102,65096,65102,65102
1,13010,13010,13010,13010
2,234495,234483,234495,234495
3,16811,16809,16811,16811
4,83322,83318,83322,83322
5,7737,7737,7737,7737
6,29079,29079,29079,29079
7,178734,178720,178734,178734


In [8]:
# lets remove videos under category 29 and 43
comments_df = comments_df[comments_df['category_id'] != 14]
comments_df = comments_df[comments_df['category_id'] != 15]

In [9]:
comments_df = comments_df.dropna()

In [10]:
comments_df.isnull().sum()

video_id        0
comment_text    0
likes           0
replies         0
category_id     0
dtype: int64

In [11]:
concat = comments_df.groupby('video_id')['comment_text'].apply(list)
concat.head()

video_id
--JinobXWPk    [I saw this wandering spider in our bathroom s...
--y69Ui8xwM    [*What was your favourite of these 10 amazing ...
-1fzGnFwz9M    [I make interesting cartoons and I need your h...
-2hRYEFiPSA    [One thing that's upset me the most about Ches...
-3AGlBYyLjo    [Where are the other interviews Gary Chandling...
Name: comment_text, dtype: object

In [12]:
concat_df = concat.reset_index()
concat_df.head()

,video_id,comment_text
0,--JinobXWPk,[I saw this wandering spider in our bathroom s...
1,--y69Ui8xwM,[*What was your favourite of these 10 amazing ...
2,-1fzGnFwz9M,[I make interesting cartoons and I need your h...
3,-2hRYEFiPSA,[One thing that's upset me the most about Ches...
4,-3AGlBYyLjo,[Where are the other interviews Gary Chandling...


In [13]:
concat_df['category_id'] = concat_df.apply(lambda row: video_id_to_category[row['video_id']], axis=1)
concat_df['category_id'] = le.transform(concat_df['category_id']) 

In [14]:
concat_df['comment_text'] = concat_df.apply(lambda row: " ".join(row['comment_text']), axis=1)

In [15]:
def clean_text(text, to_stem):
    # tokenize
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [token for token in tokens if token.isalpha()]
    # filter out stop words
    words = [w for w in words if not w in stop_words]
    # stemming words
    if to_stem:
        porter = PorterStemmer()
        words = [porter.stem(word) for word in words]
    return " ".join(words)

In [16]:
to_stem = False
concat_df['comment_text'] = videos_df.apply(lambda x: clean_and_join_title_and_tags(x, to_stem), axis=1)

In [17]:
concat_df['comment_text'][0]

'saw wander spider bathroom serious never go costa rica ca stick small ant bee mess spider im petrifi brazilian wander spider deadliest spider noth hairspray lighter could handl lol snek hey coyot episod japanes giant hornet unsaf fine scorpion fairli easi gaug know larg pincher small sting small pincher big sting wander sider also famili wolf spider wolf spider also incred fast bday octob im get book look like mini dog head ah hell na see run around lem get rpg step step love seeth glare coyot whole keel keel unlik spider one misunderstood violent vindict nope spider biolog landmin friend call brazilian wander spider idk watch video bed fall asleep think spider room sleep dude swear crazi af much respect spider one give hour long painful erect go australia find bigger danger creatur like agre spider littl condo tree mind bid dayum fewl wonder spider like alien abduct type deal go go spider friend like bruh never go believ happen last night gon na natgeo wild let part money make machin

In [18]:
vectorizer = TfidfVectorizer()
counts = vectorizer.fit_transform(concat_df['comment_text'].values)
X = counts
y = concat_df['category_id'].to_numpy()

In [19]:
def unison_shuffled_copies(a, b):
    assert a.shape[0] == b.shape[0]
    p = np.random.permutation(a.shape[0])
    return a[p], b[p]
X, y = unison_shuffled_copies(X, y)
X = np.nan_to_num(X)

#### See baseline of some models to see if task is doable

In [20]:
classifiers = {"LogisticRegression": LogisticRegression(),
               "MultinomialNB": MultinomialNB(),
               "RandomForestClassifier": RandomForestClassifier(),
               "SVC": SVC(probability=True),
               "KNeighborsClassifier": KNeighborsClassifier(),
               "DecisionTreeClassifier": DecisionTreeClassifier()}

In [21]:
accuracies = {}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM)
for classifier, model in classifiers.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracies[classifier] = accuracy_score(y_test, y_pred)

In [22]:
accuracies

{'LogisticRegression': 0.5993640699523053,
 'MultinomialNB': 0.356120826709062,
 'RandomForestClassifier': 0.3863275039745628,
 'SVC': 0.21144674085850557,
 'KNeighborsClassifier': 0.5246422893481717,
 'DecisionTreeClassifier': 0.3688394276629571}

#### Tuning KNN
Best: 0.590461 using {'algorithm': 'auto', 'n_neighbors': 20, 'weights': 'distance'}

In [27]:
# Params
algorithm = ['auto']
n_neighbors = range(17, 25)#11-2
weights = ['distance', 'uniform']

param_grid = dict(algorithm=algorithm, n_neighbors=n_neighbors, weights=weights)

model = KNeighborsClassifier()
CV = GridSearchCV(estimator=model, param_grid=param_grid, cv = 5, n_jobs=-1, verbose=3)
start_time = time.time()
result = CV.fit(X, y)

# Summarize results
print("Best: %f using %s" % (result.best_score_, result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    7.4s


Best: 0.590461 using {'algorithm': 'auto', 'n_neighbors': 20, 'weights': 'distance'}
Execution time: 27.72848868370056 ms


[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   27.6s finished


#### Tuning Random Forest
Best: 0.495707 using {'n_estimators': 88, 'random_state': 42}

In [29]:
# Params
n_estimators = range(81, 101)
random_state = [RANDOM]

param_grid = dict(n_estimators=n_estimators, random_state=random_state)

model = RandomForestClassifier()
CV = GridSearchCV(estimator=model, param_grid=param_grid, cv = 5, n_jobs=-1, verbose=3)
start_time = time.time()
result = CV.fit(X, y)

# Summarize results
print("Best: %f using %s" % (result.best_score_, result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.8min finished


Best: 0.495707 using {'n_estimators': 88, 'random_state': 42}
Execution time: 477.69982385635376 ms


#### Tuning Logistic Regression
Best: 0.658824 using {'C': 35, 'penalty': 'l2', 'random_state': 42}

In [34]:
# Params
penalty = ['l2']
C = [31, 32, 33, 34, 35, 36, 37, 38, 39]
random_state = [RANDOM]

param_grid = dict(penalty=penalty, C=C, random_state=random_state)

model = LogisticRegression()
CV = GridSearchCV(estimator=model, param_grid=param_grid, cv = 5, n_jobs=-1, verbose=3)
start_time = time.time()
result = CV.fit(X, y)

# Summarize results
print("Best: %f using %s" % (result.best_score_, result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.2min finished


Best: 0.658824 using {'C': 35, 'penalty': 'l2', 'random_state': 42}
Execution time: 136.19749450683594 ms


#### Tuning SVM
Best: 0.645151 using {'C': 21, 'gamma': 0.1, 'kernel': 'rbf', 'random_state': 42}

In [38]:
# Params
C = range(16,25)
gamma = [0.1]
kernel = ['rbf']
random_state = [RANDOM]
param_grid = dict(C=C, gamma=gamma, kernel=kernel, random_state=random_state)

model = SVC()
CV = GridSearchCV(estimator=model, param_grid=param_grid, cv = 5, n_jobs=-1, verbose=3)
start_time = time.time()
result = CV.fit(X, y)

# Summarize results
print("Best: %f using %s" % (result.best_score_, result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 12.2min finished


Best: 0.645151 using {'C': 21, 'gamma': 0.1, 'kernel': 'rbf', 'random_state': 42}
Execution time: 806.9251224994659 ms


#### Tuning Naive Bayes
Best: 0.554849 using {'alpha': 0.0, 'fit_prior': False}

In [40]:
# Params
alpha = np.linspace(0.0, 10, num=21)
fit_prior = [True, False]
param_grid = dict(alpha=alpha, fit_prior=fit_prior)

model = MultinomialNB()
CV = GridSearchCV(estimator=model, param_grid=param_grid, cv = 5, n_jobs=-1, verbose=3)
start_time = time.time()
result = CV.fit(X, y)

# Summarize results
print("Best: %f using %s" % (result.best_score_, result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    7.8s


Best: 0.554849 using {'alpha': 0.0, 'fit_prior': False}
Execution time: 12.429587125778198 ms


[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   12.2s finished
